In [1]:
1+1

2

In [17]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key=os.getenv("OPENAI_API_KEY")

In [19]:
model_client = OpenAIChatCompletionClient(model="gpt-4o", api_key=api_key)

In [22]:
assitant = AssistantAgent(name="myAssistant", model_client=model_client)
result = await assitant.run(task="What is the capital of delhi?")
print(result.messages)

[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 3, 1, 2, 29, 270680, tzinfo=datetime.timezone.utc), content='What is the capital of delhi?', type='TextMessage'), TextMessage(source='myAssistant', models_usage=RequestUsage(prompt_tokens=44, completion_tokens=24), metadata={}, created_at=datetime.datetime(2025, 7, 3, 1, 2, 30, 882283, tzinfo=datetime.timezone.utc), content='The capital of Delhi, which is a Union Territory and also a prominent city in India, is New Delhi. TERMINATE', type='TextMessage')]


In [23]:
print(result.messages[-1].content)  # Print the content of the last message

The capital of Delhi, which is a Union Territory and also a prominent city in India, is New Delhi. TERMINATE


In [13]:
result = await assitant.run(task="tell me 5+25")
print(result.messages[-1].content)  # Print the content of the last message

5 + 25 = 30. TERMINATE


In [24]:
#61
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os   

load_dotenv()
api_key=os.getenv("OPENAI_API_KEY")
model_client = OpenAIChatCompletionClient(model="gpt-4o", api_key=api_key)

assitant = AssistantAgent(
    name="HISTROY_EXPERT",
    model_client=model_client,
    description="I am a history expert, I can answer questions about history.",
    system_message="You are a history expert. You can answer questions about historical events, figures, and timelines."
    )    


async def history_expert():
    result = await assitant.run(task="What is the First  president of USA?")
    print(result.messages[-1].content)  # Print the content of the last message

await history_expert()


The first president of the United States was George Washington. He served from April 30, 1789, to March 4, 1797. Washington is often referred to as the "Father of His Country" for his pivotal role in leading the nation during its formative years, both as the commander-in-chief of the Continental Army during the American Revolutionary War and as the first president following the ratification of the U.S. Constitution.


In [25]:
#62 Tools
 
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os   

def get_weather(city:str)-> str:
    """
    Fetches the current weather for a given city.
    This is a mock function that simulates fetching weather data.
    :param city: The name of the city to get the weather for.
    :return: A string describing the weather in the city.
    """
    return f"The weather in {city} is sunny with a high of 25°C."

load_dotenv()
api_key=os.getenv("OPENAI_API_KEY")
model_client = OpenAIChatCompletionClient(model="gpt-4o", api_key=api_key)

assitant = AssistantAgent(
    name="HISTROY_EXPERT",
    model_client=model_client,
    description="I am a history expert, I can answer questions about history.",
    system_message="You are a history expert. You can answer questions about historical events, figures, and timelines.",
    tools=[get_weather]  # Register the get_weather function as a tool
    )

async def test_weather_tool():
    # Test the get_weather function
    result = await assitant.run(task="What is the weather in New York?")
    print(result.messages[-1].content)  # Print the content of the last message
 

await test_weather_tool()


The weather in New York is sunny with a high of 25°C.


In [6]:
#65 Messages


import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_core import Image as AGImage  

from autogen_ext.models.openai import OpenAIChatCompletionClient

from PIL import Image
from io import BytesIO
from dotenv import load_dotenv
import os
import requests

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable not set")
model_client = OpenAIChatCompletionClient(model="gpt-4o", api_key=api_key)


agent = AssistantAgent(
    name="textAssistant",
    model_client=model_client,
    system_message="You are a helpful assistant that can process text and images."
)

async def test_text_message():
    text_msg = TextMessage(content="capital of france?", source='user')
    result = await agent.run(task=text_msg)
    print(result.messages[-1].content)  # Print the content of the last message

await test_text_message()

The capital of France is Paris.


In [ ]:
#MutltiModalMessage

async def test_multimodal():
    response = requests.get('https://picsum.photos/id/237/200/300')
    pil_image = Image.open(BytesIO(response.content))
    ag_image= AGImage(pil_image)
    multi_modal_msg= MultiModalMessage(
        content=["What breed is this dog?",ag_image],
        source='user'
    )
    #gpt-4o supports multimodal messages
    result = await agent.run(task=multi_modal_msg)
    print(result.messages[-1].content)  # Print the content of the last message

await test_multimodal()

This dog looks like a black Labrador Retriever.


In [14]:
#67
#Running and observing the agent


from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_core import Image as AGImage  

from autogen_ext.models.openai import OpenAIChatCompletionClient

from PIL import Image
from io import BytesIO
from dotenv import load_dotenv
import requests
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

model_client = OpenAIChatCompletionClient(model="gpt-4o", api_key=api_key)

async def web_search(query:str) -> str:
    """
    Simulates a web search and returns a result.
    This is a mock function that simulates searching the web.
    :param query: The search query.
    :return: A string simulating a web search result.
    """
    return f"Search results for '{query}': [Mocked search results]"


agent = AssistantAgent(
    name="AssistantAgent",
    model_client=model_client,
    tools=[web_search],
    system_message='use tools to solve tasks'  # Register the get_weather function as a tool
)

result = await agent.run(task="find information about labrador retriever")
print(result.messages[-1].content)  # Print the content of the last message


from autogen_core import CancellationToken

async def assistant_run()-> None:
    response = await agent.on_messages(
        messages=[TextMessage(content="find information about labrador retriever", source='user')],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print('\n\n\n')
    print(response.chat_message)  # Print the content of the last message

await assistant_run()


Search results for 'Labrador Retriever information': [Mocked search results]
[ToolCallRequestEvent(source='AssistantAgent', models_usage=RequestUsage(prompt_tokens=155, completion_tokens=18), metadata={}, created_at=datetime.datetime(2025, 6, 27, 23, 44, 10, 668384, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_9FWJ7e8bY1Iss4jKpL5NE1XQ', arguments='{"query":"Labrador Retriever information"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='AssistantAgent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 27, 23, 44, 10, 670386, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content="Search results for 'Labrador Retriever information': [Mocked search results]", name='web_search', call_id='call_9FWJ7e8bY1Iss4jKpL5NE1XQ', is_error=False)], type='ToolCallExecutionEvent')]




source='AssistantAgent' models_usage=None metadata={} created_at=datetime.datetime(2025, 6, 27, 23, 44, 10, 670386, tzinfo=dateti

In [15]:
from autogen_agentchat.ui import Console

async def assistant_run_stream() -> None:
    
    await Console(
        agent.on_messages_stream(
            messages=[TextMessage(content="find information about labrador retriever", source='user')],
            cancellation_token=CancellationToken()
        ),
        output_stats= True #Enable stats printing
    )

await assistant_run_stream()

---------- ToolCallRequestEvent (AssistantAgent) ----------
[FunctionCall(id='call_TlirJh3fllcSpb3Xr8Dh5F04', arguments='{"query":"Labrador Retriever breed information"}', name='web_search')]
[Prompt tokens: 210, Completion tokens: 19]
---------- ToolCallExecutionEvent (AssistantAgent) ----------
[FunctionExecutionResult(content="Search results for 'Labrador Retriever breed information': [Mocked search results]", name='web_search', call_id='call_TlirJh3fllcSpb3Xr8Dh5F04', is_error=False)]
---------- AssistantAgent ----------
Search results for 'Labrador Retriever breed information': [Mocked search results]
---------- Summary ----------
Number of inner messages: 2
Total prompt tokens: 210
Total completion tokens: 19
Duration: 2.33 seconds


In [26]:
#68
#Structured output
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_core import Image as AGImage  
from autogen_ext.models.openai import OpenAIChatCompletionClient
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os

class PlanetInfo(BaseModel):
    name: str
    color:str
    distance_miles:int

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=api_key,
    response_model = PlanetInfo
    )

agent = AssistantAgent(
    name="PlanetInfoAgent",
    model_client=model_client,
    system_message="You are an expert in planetary science. "
)

async def test_Structured_output():
    task = TextMessage(content="What is the color and distance of Mars from Earth in miles?", source='user')
    result = await agent.run(task=task)
    structured_outcome= result.messages[-1].content
    print(structured_outcome)  # Print the structured output

await test_Structured_output()

Mars is often referred to as the "Red Planet" due to its reddish appearance, which is primarily caused by iron oxide, or rust, on its surface.

The distance between Mars and Earth varies significantly due to their elliptical orbits. On average, Mars is about 140 million miles (225 million kilometers) from Earth. However, at its closest approach, called opposition, Mars can be as close as approximately 34 million miles (55 million kilometers) to Earth. At its farthest, when the two planets are on opposite sides of the Sun, the distance can be more than 250 million miles (400 million kilometers). Distance fluctuates over a two-year cycle, roughly aligning with Mars' opposition to Earth every 26 months.


In [ ]:
# Multi Agent Systems

#68
#Structured output
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage, MultiModalMessage
from autogen_core import Image as AGImage  
from autogen_ext.models.openai import OpenAIChatCompletionClient
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os 

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=api_key
    )

#Single Agent Approach
story_agent = AssistantAgent(
    name="Story_Agent",
    model_client=model_client,
    system_message="You are an expert in creating stories. "
)

async def test_simple_agent():
    task = TextMessage(content="Write a short story about a brave knight.", source='user')
    result = await story_agent.run(task=task)
    print(result.messages[-1].content)  # Print the content of the last message

await test_simple_agent()

In the heart of the verdant kingdom of Elloria, nestled between sprawling forests and glistening rivers, there lay a quaint village known as Elderglen. The villagers were simple folk, tending to their sheep and harvesting golden fields of wheat. Yet, despite its idyllic landscape, Elderglen was shrouded in a shadow of fear, for in the cliffs above where forests thickened and mists never lifted, there resided a fearsome dragon named Tharion.

Tharion was no ordinary dragon; his scales shimmered like a fiery sunrise, his eyes burned with the intensity of embers, and his roar could shake the very ground itself. He was the terror of the realm, descending from his lofty perch every year to claim tribute from the frightened villagers.

In Elderglen, tales of the dragon's wrath were spoken in hushed voices, and the village council reluctantly paid the yearly tribute of livestock to keep the beast at bay. But with each passing year, the dragon's demands grew, leaving the villagers little to sp

In [27]:
#multi_agent_system

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
 
from dotenv import load_dotenv
import os 

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=api_key
    )

#Single Agent Approach
plot_agent = AssistantAgent(
    name="PLOT_AGENT",
    model_client=model_client,
    system_message="You create enagaging plot for stories. Focus on the knights journey "
)

character_agent = AssistantAgent(
    name="Character_writer",
    model_client=model_client,
    system_message="You develop characters. Describe the knight and dragon in detail including their motivations and personalities."
)

ending_agent = AssistantAgent(
    name="Ending_writer",
    model_client=model_client,
    system_message="You create a compelling ending for the story. Ensure it is satisfying and ties up all loose ends."
)

team_agents= RoundRobinGroupChat(
    [plot_agent, character_agent, ending_agent],
    max_turns=3,  # Maximum number of rounds for the conversation
)

async def test_team_agents():
    task = TextMessage(content="Write a short story about a brave knight. keep it under 500 words",
                        source='user')
    result = await team_agents.run(task=task)
    for each_agent_message in result.messages:
        print(f"{each_agent_message.source }:\n {each_agent_message.content}\n")
    print(result.messages[-1].content)  # Print the content of the last message


await test_team_agents()



user:
 Write a short story about a brave knight. keep it under 500 words

PLOT_AGENT:
 In the heart of the Velorian Kingdom, where emerald hills touched the sky and the whisper of ancient legends danced upon the wind, there lived a valiant knight named Sir Aldric. Known throughout the realm for his unwavering courage and noble heart, Aldric carried a legacy passed down from the great warriors before him. His armor gleamed like the morning sun, reflecting his solemn vow to protect the weak and uphold justice.

One crisp autumn morning, as the mist curled around the forest like a secret cloak, a proclamation echoed through the stone corridors of Velorian Castle. Queen Elara issued a summons to all knights: the cursed beast, Fenrir, had been sighted near the village of Eldergrove, leaving a trail of destruction. With a roar of anticipation from his fellow knights, Aldric volunteered to embark on this quest to bring peace back to the troubled hearts of his people.

With his steed, Bramble,

In [ ]:
#comparison table

In [2]:
#observability
#71
 
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult
from autogen_agentchat.ui import Console
from dotenv import load_dotenv

import os 

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=api_key
    )

#Single Agent Approach
plot_agent = AssistantAgent(
    name="PLOT_AGENT",
    model_client=model_client,
    system_message="You create enagaging plot for stories. Focus on the knights journey. Complete in 10 words "
)

character_agent = AssistantAgent(
    name="Character_writer",
    model_client=model_client,
    system_message="You develop characters. Describe the knight and dragon in detail including their motivations and personalities.  Complete in 10 words"
)

ending_agent = AssistantAgent(
    name="Ending_writer",
    model_client=model_client,
    system_message="You create a compelling ending for the story. Ensure it is satisfying and ties up all loose ends. Complete in 10 words"
)

team_agents= RoundRobinGroupChat(
    [plot_agent, character_agent, ending_agent],
    max_turns=3,  # Maximum number of rounds for the conversation
)

async for message in team_agents.run_stream(task="write a poem on the spring season"):
    #print(message)
    #print(type(message))
    if isinstance(message, TaskResult):
        print(f"Stop reason: {message.stop_reason}")
    else:
        print((message))

await Console(team_agents.run_stream(task="write a poem on the spring season"))  
  



source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 6, 30, 3, 39, 8, 258325, tzinfo=datetime.timezone.utc) content='write a poem on the spring season' type='TextMessage'
source='PLOT_AGENT' models_usage=RequestUsage(prompt_tokens=40, completion_tokens=15) metadata={} created_at=datetime.datetime(2025, 6, 30, 3, 39, 9, 493845, tzinfo=datetime.timezone.utc) content='Blossoms bloom, gentle breeze whispers, nature awakens with grace.' type='TextMessage'
source='Character_writer' models_usage=RequestUsage(prompt_tokens=64, completion_tokens=37) metadata={} created_at=datetime.datetime(2025, 6, 30, 3, 39, 10, 757769, tzinfo=datetime.timezone.utc) content='Knight: Brave, honorable, seeks justice; driven by chivalry, loyalty, compassion.\n\nDragon: Fierce, wise, guards treasure; motivated by survival, solitude, legacy.' type='TextMessage'
source='Ending_writer' models_usage=RequestUsage(prompt_tokens=110, completion_tokens=13) metadata={} created_at=datetime.datetime

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 30, 3, 39, 11, 663849, tzinfo=datetime.timezone.utc), content='write a poem on the spring season', type='TextMessage'), TextMessage(source='PLOT_AGENT', models_usage=RequestUsage(prompt_tokens=135, completion_tokens=16), metadata={}, created_at=datetime.datetime(2025, 6, 30, 3, 39, 32, 944938, tzinfo=datetime.timezone.utc), content="Spring arrives, colors burst, life renews, warmth embraces earth's heart.", type='TextMessage'), TextMessage(source='Character_writer', models_usage=RequestUsage(prompt_tokens=160, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 6, 30, 3, 39, 54, 100118, tzinfo=datetime.timezone.utc), content='Emerald buds unfurl, blossoms paint fields, joyous songs fill skies.', type='TextMessage'), TextMessage(source='Ending_writer', models_usage=RequestUsage(prompt_tokens=184, completion_tokens=14), metadata={}, created_at=datetime.

In [1]:
#Team Operations: Reset, Stop, Resume and Abort
#72
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult
from autogen_agentchat.ui import Console
from dotenv import load_dotenv
import os 

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=api_key
    )

add_1_agent_first = AssistantAgent(
    name="add_1_agent_first",
    model_client=model_client,
    system_message="Add 1 to the number , first number is 0. Give result as output"
)

add_1_agent_second = AssistantAgent(
    name="add_1_agent_second",
    model_client=model_client,
    system_message="Add 1 to the output. Give result as output."
)

add_1_agent_third = AssistantAgent(
    name="add_1_agent_third",
    model_client=model_client,
    system_message="Add 1 to the output. Give result as output."
)

team_agents= RoundRobinGroupChat(
    [add_1_agent_first, add_1_agent_second, add_1_agent_third],  
    max_turns=3,  # Maximum number of rounds for the conversation
)

await Console(team_agents.run_stream())


---------- TextMessage (add_1_agent_first) ----------
1
---------- TextMessage (add_1_agent_second) ----------
2
---------- TextMessage (add_1_agent_third) ----------
3


TaskResult(messages=[TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 6, 30, 21, 18, 28, 308405, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 6, 30, 21, 18, 29, 213617, tzinfo=datetime.timezone.utc), content='2', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=39, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 6, 30, 21, 18, 30, 456152, tzinfo=datetime.timezone.utc), content='3', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

In [2]:
await Console(team_agents.run_stream(task="What is the largest number in the result"))

---------- TextMessage (user) ----------
What is the largest number in the result
---------- TextMessage (add_1_agent_first) ----------
The largest number in the sequence you provided is 3.
---------- TextMessage (add_1_agent_second) ----------
The largest number in the sequence you provided is 3. Adding 1 to 3 gives 4.
---------- TextMessage (add_1_agent_third) ----------
The largest number in the result is 4.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 30, 21, 23, 32, 382834, tzinfo=datetime.timezone.utc), content='What is the largest number in the result', type='TextMessage'), TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=63, completion_tokens=12), metadata={}, created_at=datetime.datetime(2025, 6, 30, 21, 23, 33, 294092, tzinfo=datetime.timezone.utc), content='The largest number in the sequence you provided is 3.', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=79, completion_tokens=22), metadata={}, created_at=datetime.datetime(2025, 6, 30, 21, 23, 33, 912675, tzinfo=datetime.timezone.utc), content='The largest number in the sequence you provided is 3. Adding 1 to 3 gives 4.', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=109, completion_tokens=10), metadata={}, created_at=

In [10]:
#Resuming a team- Continues from top with final result
await Console(team_agents.run_stream())

---------- TextMessage (add_1_agent_first) ----------
The result of adding 1 to the number 3 is 4.
---------- TextMessage (add_1_agent_second) ----------
The result of adding 1 to the number 4 is 5.
---------- TextMessage (add_1_agent_third) ----------
The result of adding 1 to the number 5 is 6.


TaskResult(messages=[TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=92, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 6, 30, 17, 9, 53, 148951, tzinfo=datetime.timezone.utc), content='The result of adding 1 to the number 3 is 4.', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=111, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 6, 30, 17, 10, 14, 244532, tzinfo=datetime.timezone.utc), content='The result of adding 1 to the number 4 is 5.', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=134, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 6, 30, 17, 10, 35, 242120, tzinfo=datetime.timezone.utc), content='The result of adding 1 to the number 5 is 6.', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

In [4]:
#reset the team
await team_agents.reset()

#running after reset
await Console(team_agents.run_stream())

---------- TextMessage (add_1_agent_first) ----------
1
---------- TextMessage (add_1_agent_second) ----------
2
---------- TextMessage (add_1_agent_third) ----------
3


TaskResult(messages=[TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 6, 30, 21, 25, 38, 536110, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 6, 30, 21, 25, 39, 290609, tzinfo=datetime.timezone.utc), content='2', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=39, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 6, 30, 21, 25, 40, 828760, tzinfo=datetime.timezone.utc), content='3', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

In [7]:
#73 Terminations in microsoft autogen
#Agents can run forever, so we need to terminate them
#use terminiation conidition to stop the agents

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import MaxMessageTermination 
from dotenv import load_dotenv
import os 

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=api_key
    )

termination_condition_max = MaxMessageTermination(max_messages=5)


add_1_agent_first = AssistantAgent(
    name="add_1_agent_first",
    model_client=model_client,
    system_message="Add 1 to the number , first number is 0. Give result as output"
)

add_1_agent_second = AssistantAgent(
    name="add_1_agent_second",
    model_client=model_client,
    system_message="Add 1 to the output. Give result as output."
)

add_1_agent_third = AssistantAgent(
    name="add_1_agent_third",
    model_client=model_client,
    system_message="Add 1 to the output. Give result as output."
)

team_agents= RoundRobinGroupChat(
    [add_1_agent_first, add_1_agent_second, add_1_agent_third],
    termination_condition=termination_condition_max,  # Set the termination condition
)

await Console(team_agents.run_stream())



---------- TextMessage (add_1_agent_first) ----------
1
---------- TextMessage (add_1_agent_second) ----------
The result is 2.
---------- TextMessage (add_1_agent_third) ----------
The result is 3.
---------- TextMessage (add_1_agent_first) ----------
The result is 4.
---------- TextMessage (add_1_agent_second) ----------
The result is 5.


TaskResult(messages=[TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 1, 1, 52, 17, 146720, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=6), metadata={}, created_at=datetime.datetime(2025, 7, 1, 1, 52, 17, 518778, tzinfo=datetime.timezone.utc), content='The result is 2.', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=44, completion_tokens=6), metadata={}, created_at=datetime.datetime(2025, 7, 1, 1, 52, 18, 40418, tzinfo=datetime.timezone.utc), content='The result is 3.', type='TextMessage'), TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=60, completion_tokens=6), metadata={}, created_at=datetime.datetime(2025, 7, 1, 1, 52, 39, 288570, tzinfo=datetime.timezone.utc), content

In [9]:
#external termination

#74

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import MaxMessageTermination, ExternalTermination
from dotenv import load_dotenv
import os 
import asyncio

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=api_key
    )

agent1 = AssistantAgent(
    name="story_writer",
    model_client=model_client,
    system_message="Give the story about a brave knight. The story should be engaging and inspiring.Keep it short within 30 words"
)

agent2 = AssistantAgent(
    name="story_critic",
    model_client=model_client,
    system_message="Continue the story and ciritc it with feedback. Keep it short within 30 words"
)


ext_termination = ExternalTermination()

team_agents= RoundRobinGroupChat(
    [agent1, agent2],
    termination_condition=ext_termination,  # Set the termination condition
)

run = asyncio.create_task(
    Console(team_agents.run_stream(task="write a story about a brave knight"))
)

await asyncio.sleep(2)  # Let the agents run for a while

ext_termination.set()  # Terminate the agents externally
await run

---------- TextMessage (user) ----------
write a story about a brave knight
---------- TextMessage (story_writer) ----------
Sir Elric, driven by courage, ventured into dragon-guarded realms, defeating beasts with valor. His bravery inspired his kingdom, proving that true heroism always lights the path to victory.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 1, 2, 22, 33, 722469, tzinfo=datetime.timezone.utc), content='write a story about a brave knight', type='TextMessage'), TextMessage(source='story_writer', models_usage=RequestUsage(prompt_tokens=42, completion_tokens=39), metadata={}, created_at=datetime.datetime(2025, 7, 1, 2, 22, 36, 118030, tzinfo=datetime.timezone.utc), content='Sir Elric, driven by courage, ventured into dragon-guarded realms, defeating beasts with valor. His bravery inspired his kingdom, proving that true heroism always lights the path to victory.', type='TextMessage')], stop_reason='External termination requested')

In [ ]:
#75
#Aborting a team of agents


from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import MaxMessageTermination, ExternalTermination
from dotenv import load_dotenv
import os 
import asyncio

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
cancellation_token= CancellationToken()

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=api_key
    )

agent1 = AssistantAgent(
    name="solo_poet",
    model_client=model_client,
    system_message="You are a poet. Write a short poem on Moon and end with Moonlight. Keep it short within 30 words"
)
termin

In [15]:
#77
#managing state

from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import MaxMessageTermination, ExternalTermination
from dotenv import load_dotenv
import os 
import asyncio

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
cancellation_token= CancellationToken()

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=api_key
    )


assistant = AssistantAgent(
    name="Assistant",
    model_client=model_client,
    system_message="You are helpful assistant. You can answer questions about various topics. "
)

response = await assistant.on_messages(
    messages=[TextMessage(content="Can you help me with a travel iternary of France from May to June and keep it concise", source='user')],
    cancellation_token=cancellation_token,
)

print(response.chat_message.content)  # Print the content of the last message

agent_state = await assistant.save_state()

print(agent_state)  # Print the saved state

new_assistant_agent = AssistantAgent(
    name="Assistant_2",
    model_client=model_client,
    system_message="You are helpful assistant who can answer questions and provide information about various topics. "
)

await new_assistant_agent.load_state(agent_state)

response = await new_assistant_agent.on_messages(
    messages=[TextMessage(content="can you tell me what was the last discussion about?", source='user')],
    cancellation_token=cancellation_token,
)

print(f"\n{"state content :: "}\n{response.chat_message.content}\n")  # Print the content of the last message

Certainly! Here’s a concise travel itinerary for a trip to France from May through June, covering key highlights and experiences:

**Week 1: Paris**
- **Days 1-3:** Explore Paris
  - Visit iconic sites: Eiffel Tower, Louvre Museum, Notre-Dame Cathedral.
  - Stroll through Montmartre and visit Sacré-Cœur.
  - Relax in Luxembourg Gardens.

- **Days 4-5:** Day Trips
  - Day trip to Versailles: Tour the opulent Palace and gardens.
  - Optional day trip to Giverny: See Monet’s gardens.

**Week 2: Normandy & Loire Valley**
- **Day 6:** Travel to Normandy
  - Visit Rouen and its historic cathedral.

- **Days 7-8:** D-Day Beaches & Mont Saint-Michel
  - Explore the D-Day Landing Beaches.
  - Visit Mont Saint-Michel, an island commune.

- **Days 9-10:** Loire Valley
  - Tour the châteaux: Chambord, Chenonceau, and Amboise.
  - Taste local wines and enjoy the countryside.

**Week 3: Bordeaux & Dordogne Region**
- **Day 11:** Travel to Bordeaux
  - Explore the city’s architecture and wine bars.



In [2]:
#managing state in teams
#78

from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import MaxMessageTermination, ExternalTermination
from dotenv import load_dotenv
import os 
import asyncio

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
cancellation_token= CancellationToken()

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=api_key 
    )


agent_1 = AssistantAgent(
    name="Writer_1",
    model_client=model_client,
    system_message="You are helpful assistant. Give the output in less than 30 words"
)


agent_2 = AssistantAgent(
    name="Writer_2",
    model_client=model_client,
    system_message="You are Classic Writer. Give the output in less than 30 words"
)

terminationCondition = MaxMessageTermination(max_messages=3)

agent_team= RoundRobinGroupChat(participants=[agent_1, agent_2], termination_condition=terminationCondition)

stream = agent_team.run_stream(task="Write a poem about the sea in 3 lines")

await Console(stream) 


team_state = await agent_team.save_state()
print("Team State saved successfully.")
print(team_state)  # Print the saved state

---------- TextMessage (user) ----------
Write a poem about the sea in 3 lines
---------- TextMessage (Writer_1) ----------
Endless waves dance and gleam,  
Salt-kissed air sings timeless tales,  
Deep horizon dreams untold.
---------- TextMessage (Writer_2) ----------
Ocean whispers call softly,  
Majestic tides weave secrets,  
Blue cradle of endless hope.
Team State saved successfully.
{'type': 'TeamState', 'version': '1.0.0', 'agent_states': {'Writer_1': {'type': 'ChatAgentContainerState', 'version': '1.0.0', 'agent_state': {'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Write a poem about the sea in 3 lines', 'source': 'user', 'type': 'UserMessage'}, {'content': 'Endless waves dance and gleam,  \nSalt-kissed air sings timeless tales,  \nDeep horizon dreams untold.', 'thought': None, 'source': 'Writer_1', 'type': 'AssistantMessage'}]}}, 'message_buffer': [{'id': '8afcaa7b-01a9-4e59-a183-a2081ee4a3a4', 'source': 'Writer_2', 'models_usage

In [4]:


team_state = await agent_team.save_state()
print("Team State saved successfully.")
print(team_state)  # Print the saved state

Team State saved successfully.
{'type': 'TeamState', 'version': '1.0.0', 'agent_states': {'Writer_1': {'type': 'ChatAgentContainerState', 'version': '1.0.0', 'agent_state': {'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Write a poem about the sea in 3 lines', 'source': 'user', 'type': 'UserMessage'}, {'content': 'Endless waves in dance and play,  \nWhisper secrets in salty spray,  \nSky and ocean blend, night meets day.', 'thought': None, 'source': 'Writer_1', 'type': 'AssistantMessage'}]}}, 'message_buffer': [{'source': 'Writer_2', 'models_usage': {'prompt_tokens': 70, 'completion_tokens': 22}, 'metadata': {}, 'created_at': datetime.datetime(2025, 7, 14, 17, 45, 3, 826501, tzinfo=datetime.timezone.utc), 'content': 'Vast blue whispers endless,  \nSilvery tides kiss the shore,  \nDreams afloat, forevermore.', 'type': 'TextMessage'}]}, 'Writer_2': {'type': 'ChatAgentContainerState', 'version': '1.0.0', 'agent_state': {'type': 'AssistantAgent

In [5]:
await agent_team.load_state(team_state) #helps in remembering the last conversation
stream = agent_team.run_stream(task="What was the last line of poem you wrote?")
await Console(stream)  # Print the content of the last message



---------- TextMessage (user) ----------
What was the last line of poem you wrote?


---------- TextMessage (Writer_1) ----------
Dreams afloat, forevermore.
---------- TextMessage (Writer_2) ----------
Dreams afloat, forevermore.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 14, 17, 45, 23, 317453, tzinfo=datetime.timezone.utc), content='What was the last line of poem you wrote?', type='TextMessage'), TextMessage(source='Writer_1', models_usage=RequestUsage(prompt_tokens=111, completion_tokens=7), metadata={}, created_at=datetime.datetime(2025, 7, 14, 17, 45, 24, 836236, tzinfo=datetime.timezone.utc), content='Dreams afloat, forevermore.', type='TextMessage'), TextMessage(source='Writer_2', models_usage=RequestUsage(prompt_tokens=126, completion_tokens=7), metadata={}, created_at=datetime.datetime(2025, 7, 14, 17, 45, 26, 289179, tzinfo=datetime.timezone.utc), content='Dreams afloat, forevermore.', type='TextMessage')], stop_reason='Maximum number of messages 3 reached, current message count: 3')

In [6]:
team_state #dict type can be persisted in a database or file system

{'type': 'TeamState',
 'version': '1.0.0',
 'agent_states': {'Writer_1': {'type': 'ChatAgentContainerState',
   'version': '1.0.0',
   'agent_state': {'type': 'AssistantAgentState',
    'version': '1.0.0',
    'llm_context': {'messages': [{'content': 'Write a poem about the sea in 3 lines',
       'source': 'user',
       'type': 'UserMessage'},
      {'content': 'Endless waves in dance and play,  \nWhisper secrets in salty spray,  \nSky and ocean blend, night meets day.',
       'thought': None,
       'source': 'Writer_1',
       'type': 'AssistantMessage'}]}},
   'message_buffer': [{'source': 'Writer_2',
     'models_usage': {'prompt_tokens': 70, 'completion_tokens': 22},
     'metadata': {},
     'created_at': datetime.datetime(2025, 7, 14, 17, 45, 3, 826501, tzinfo=datetime.timezone.utc),
     'content': 'Vast blue whispers endless,  \nSilvery tides kiss the shore,  \nDreams afloat, forevermore.',
     'type': 'TextMessage'}]},
  'Writer_2': {'type': 'ChatAgentContainerState',
   

In [16]:
import json

with open('team_state.json', 'w') as f:
   team_state_file= json.dump(team_state, f, default=str)  # Use default=str to handle non-serializable types

In [1]:
#saving and loading state in teams from file - File stored in above

#managing state in teams
#79

from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import MaxMessageTermination, ExternalTermination
from dotenv import load_dotenv
import os 
import asyncio

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
 
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=api_key 
    )


agent_1 = AssistantAgent(
    name="Writer_1",
    model_client=model_client,
    system_message="You are helpful assistant. Give the output in less than 30 words"
)


agent_2 = AssistantAgent(
    name="Writer_2",
    model_client=model_client,
    system_message="You are Classic Writer. Give the output in less than 30 words"
)

terminationCondition = MaxMessageTermination(max_messages=3)

new_agent_team= RoundRobinGroupChat(participants=[agent_1, agent_2], termination_condition=terminationCondition)
#new Agent team getts state
new_agent_team.load_state(team_state_file)  # Load the state from the file
stream = new_agent_team.run_stream(task="What was the line you wrote last?")
await Console(stream)  # Print the content of the last message
 

NameError: name 'team_state_file' is not defined

In [28]:
#Human in the loop
#feedback during a run 


from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import MaxMessageTermination, ExternalTermination, TextMentionTermination
from dotenv import load_dotenv
import os 
import asyncio

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
 
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=api_key 
    )


myPA_agent = AssistantAgent(
    name="custom_PA",
    model_client=model_client,
    system_message="You are helpful assistant. You can answer questions about various topics. "
)

user_proxy_agent = UserProxyAgent(
    name='UserProxy',
    description="You are a user proxy agent that represents the user in the conversation. ",
    input_func= input
)

text_terminationCondition = TextMentionTermination(['APPROVED', 'DONE', 'YES','OK','GOTCHA'])
# #text_terminationCondition = TextMentionTermination(
#     ['APPROVED', 'DONE', 'YES', 'OK', 'GOTCHA'],
#     case_insensitive=True
# )

team = RoundRobinGroupChat(
    participants=[myPA_agent, user_proxy_agent],
    termination_condition=text_terminationCondition  # Set the termination condition
)

stream = team.run_stream(task="Can you help with a short plan for Agentic AI interview preparation?")

async def main():
    await Console(stream)

if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
#Autogen studio
#Create env
#